In [10]:
from google.cloud import bigquery
import pandas as pd

from google.oauth2 import service_account

key_path = "crunchbase-startups-b791b011b22c.json"

credentials = service_account.Credentials.from_service_account_file(
key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

client = bigquery.Client(credentials=credentials, project=credentials.project_id,)

sql = """
    SELECT *
    FROM `crunchbase-startups.startups.join_table`
"""

df = client.query(sql).to_dataframe()

In [11]:
# Data Processing
import pandas as pd
import numpy as np

# Modelling
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint

# Tree Visualisation
from sklearn.tree import export_graphviz
from IPython.display import Image
import graphviz

In [33]:
features = ['investment_rounds', 'funding_rounds', 'funding_total_usd', 'milestones', 'lifetime', \
            'num_missing_features', 'has_website', 'has_twitter', 'description_length', 'overview_length', \
            'overview_class_field1', 'overview_class_field2', 'overview_class_field3', 'overview_class_field4', 'investment_period', \
            'funding_period', 'milestone_period', 'investment_rate', 'funding_rate', 'milestone_rate', \
            'inverse_dt_since_last_investment', 'inverse_dt_since_last_funding', 'inverse_dt_since_last_milestone', 'inverse_data_creation_gap', 'update_gap', \
            'category_encoded_op', 'category_encoded_ac', 'category_encoded_ip', 'category_encoded_cl', 'country_encoded_op', \
            'country_encoded_ac', 'country_encoded_ip', 'country_encoded_cl', 'state_encoded_op', 'state_encoded_ac', \
            'state_encoded_ip', 'state_encoded_cl']

In [34]:
labels = ['status_operating', 'status_acquired', 'status_ipo', 'status_closed']

In [35]:
X = df[features]
Y = df[labels]

In [36]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [37]:
rf = RandomForestClassifier()
rf.fit(X_train, Y_train)

RandomForestClassifier()

In [38]:
Y_pred = rf.predict(X_test)
accuracy = accuracy_score(np.array(Y_test).astype(float), Y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9407409292147982
